# Grouping and Summarising Dataframes


Grouping analysis :
1. **Splitting** the data into groups 
2. **Applying** a function to each group 
3. **Combining** the results into a data structure 

Let's work through some examples.

### Problem : Where am I incurring heavy losses?

* Which customer segments are the least profitable?
* Which product categories and sub-categories are the least profitable?
* Customers in which geographic region cause the most losses?
* Etc.

In [5]:
# Loading libraries and files
import numpy as np
import pandas as pd

market_df = pd.read_csv("global_sales_data/market_fact.csv")
customer_df = pd.read_csv("global_sales_data/cust_dimen.csv")
product_df = pd.read_csv("global_sales_data/prod_dimen.csv")
shipping_df = pd.read_csv("global_sales_data/shipping_dimen.csv")
orders_df = pd.read_csv("global_sales_data/orders_dimen.csv")

In [6]:
# Merging the dataframes one by one
df_1 = pd.merge(market_df, customer_df, how='inner', on='Cust_id')
df_2 = pd.merge(df_1, product_df, how='inner', on='Prod_id')
df_3 = pd.merge(df_2, shipping_df, how='inner', on='Ship_id')
master_df = pd.merge(df_3, orders_df, how='inner', on='Ord_id')

master_df.head()

Ord_id  Prod_id   Ship_id    Cust_id    Sales  Discount  Order_Quantity  \
0  Ord_5446  Prod_16  SHP_7609  Cust_1818   136.81      0.01              23   
1  Ord_5446   Prod_4  SHP_7610  Cust_1818  4701.69      0.00              26   
2  Ord_5446   Prod_6  SHP_7608  Cust_1818   164.02      0.03              23   
3  Ord_2978  Prod_16  SHP_4112  Cust_1088   305.05      0.04              27   
4  Ord_5484  Prod_16  SHP_7663  Cust_1820   322.82      0.05              35   

    Profit  Shipping_Cost  Product_Base_Margin  ...   Region Customer_Segment  \
0   -30.51           3.60                 0.56  ...     WEST        CORPORATE   
1  1148.90           2.50                 0.59  ...     WEST        CORPORATE   
2   -47.64           6.15                 0.37  ...     WEST        CORPORATE   
3    23.12           3.37                 0.57  ...  ONTARIO      HOME OFFICE   
4   -17.58           3.98                 0.56  ...     WEST         CONSUMER   

  Product_Category           Product_Sub_Category Order_ID_x    Ship_Mode  \
0  OFFICE SUPPLIES  SCISSORS, RULERS AND TRIMMERS      36262  REGULAR AIR   
1       TECHNOLOGY   TELEPHONES AND COMMUNICATION      36262  EXPRESS AIR   
2  OFFICE SUPPLIES                          PAPER      36262  EXPRESS AIR   
3  OFFICE SUPPLIES  SCISSORS, RULERS AND TRIMMERS      37863  REGULAR AIR   
4  OFFICE SUPPLIES  SCISSORS, RULERS AND TRIMMERS      53026  REGULAR AIR   

    Ship_Date Order_ID_y  Order_Date  Order_Priority  
0  28-07-2010      36262  27-07-2010   NOT SPECIFIED  
1  27-07-2010      36262  27-07-2010   NOT SPECIFIED  
2  28-07-2010      36262  27-07-2010   NOT SPECIFIED  
3  26-02-2011      37863  24-02-2011            HIGH  
4  03-03-2012      53026  26-02-2012             LOW  

[5 rows x 22 columns]

#### Step 1. Grouping using ```df.groupby()```

In [16]:
# Which customer segments are the least profitable?
df_by_segment = master_df.groupby("Customer_Segment")
df_by_segment


#### Step 2. Applying a Function

In [17]:
df_by_segment['Profit'].sum()

Customer_Segment
CONSUMER          287959.94
CORPORATE         599746.00
HOME OFFICE       318354.03
SMALL BUSINESS    315708.01
Name: Profit, dtype: float64

In [18]:
df_by_segment.Profit.sum().sort_values(ascending=False)

Customer_Segment
CORPORATE         599746.00
HOME OFFICE       318354.03
SMALL BUSINESS    315708.01
CONSUMER          287959.94
Name: Profit, dtype: float64

#### Step 3. Combining the results into a Data Structure

In [19]:
pd.DataFrame(df_by_segment.Profit.sum())

Profit
Customer_Segment           
CONSUMER          287959.94
CORPORATE         599746.00
HOME OFFICE       318354.03
SMALL BUSINESS    315708.01

### Which product categories are least profitable?



#### Step 1. Group by product category

In [24]:
by_product_cat = master_df.groupby('Product_Category')

#### Step 2. This time lets compare avg Profit

In [25]:
by_product_cat['Profit'].mean()

Product_Category
FURNITURE           68.116607
OFFICE SUPPLIES    112.369074
TECHNOLOGY         429.207516
Name: Profit, dtype: float64

### Which product categories and sub categories are the least profitable ?

In [27]:
by_product_cat_subcat =master_df.groupby(['Product_Category','Product_Sub_Category'])

In [28]:
by_product_cat_subcat['Profit'].mean()

Product_Category  Product_Sub_Category          
FURNITURE         BOOKCASES                         -177.683228
                  CHAIRS & CHAIRMATS                 387.693601
                  OFFICE FURNISHINGS                 127.446612
                  TABLES                            -274.411357
OFFICE SUPPLIES   APPLIANCES                         223.866498
                  BINDERS AND BINDER ACCESSORIES     335.970918
                  ENVELOPES                          195.864228
                  LABELS                              47.490174
                  PAPER                               36.949551
                  PENS & ART SUPPLIES                 11.950679
                  RUBBER BANDS                        -0.573575
                  SCISSORS, RULERS AND TRIMMERS      -54.161458
                  STORAGE & ORGANIZATION              12.205403
TECHNOLOGY        COMPUTER PERIPHERALS               124.389815
                  COPIERS AND FAX                   192

#### Apply multiple functions

In [30]:
by_product_cat['Profit'].describe()

count        mean          std       min       25%     50%  \
Product_Category                                                                
FURNITURE         1724.0   68.116607  1112.923257 -11053.60 -281.3550 -14.250   
OFFICE SUPPLIES   4610.0  112.369074   744.617939  -2175.09  -57.0225  -3.845   
TECHNOLOGY        2065.0  429.207516  1863.208375 -14140.70  -88.9400  66.220   

                       75%       max  
Product_Category                      
FURNITURE         187.1600   8614.79  
OFFICE SUPPLIES    56.9475  11535.28  
TECHNOLOGY        561.1300  27220.69

In [31]:
by_product_cat['Profit'].count()

Product_Category
FURNITURE          1724
OFFICE SUPPLIES    4610
TECHNOLOGY         2065
Name: Profit, dtype: int64

### Customers in which geographic region are the least profitable?

In [32]:
master_df.groupby('Region').Profit.mean()

Region
ATLANTIC                 221.259870
NORTHWEST TERRITORIES    255.464670
NUNAVUT                   35.963418
ONTARIO                  189.960865
PRARIE                   188.253294
QUEBEC                   179.803649
WEST                     149.175595
YUKON                    136.253155
Name: Profit, dtype: float64

### Calculate the Sales across each region as a percentage of total sales

In [39]:
(master_df.groupby('Region').Sales.sum() /sum(master_df['Sales']))*100

Region
ATLANTIC                 13.504305
NORTHWEST TERRITORIES     5.369193
NUNAVUT                   0.780233
ONTARIO                  20.536970
PRARIE                   19.022396
QUEBEC                   10.124936
WEST                     24.119372
YUKON                     6.542595
Name: Sales, dtype: float64

## Thank You 